In [0]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import pyspark.sql.functions as F
from pyspark.sql.functions import col

# Respondendo às perguntas

# Qual o padrão de variação das estrelas ao longo do tempo

df_fato = spark.table("fato_medicao")
df_estrela = spark.table("dim_estrela")
df_data = spark.table("dim_data")
df_incerteza = spark.table("dim_incerteza")

df_consulta = (
    df_fato.alias("f")
    .join(df_estrela.alias("e"), col("f.id_estrela") == col("e.id"))
    .join(df_data.alias("d"), col("f.id_data") == col("d.id"))
    .join(df_incerteza.alias("i"), col("f.id_incerteza") == col("i.id"))
    .select("e.nome", "d.data", "f.magnitude")
    .orderBy("data")
    )

pdf = df_consulta.toPandas()

group_by_star = pdf.groupby('nome')['magnitude'].agg(
    desvio_padrao='std',
    media='mean',
    menor_brilho='max', # quanto maior a magnitude menor o brilho da estrela
    maior_brilho='min'  # quanto menor a magnitude mair o brilho da estrela
).reset_index().round(3)

display(group_by_star.sort_values('desvio_padrao', ascending=False))


In [0]:
# Sazonalidade das observações

from pyspark.sql import functions as F
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

estrelas_alvo = ['Betelgeuse', 'Eta Carinae', 'Algol']

df_consulta = (
    spark.table("fato_medicao").alias("f")
    .join(spark.table("dim_estrela").alias("e"), F.col("f.id_estrela") == F.col("e.id"))
    .join(spark.table("dim_data").alias("d"), F.col("f.id_data") == F.col("d.id"))
    .filter(F.col("e.nome").isin(estrelas_alvo))
    .select(
        F.month(F.col("d.data")).alias("mes_num"),
        F.col("e.nome").alias("Estrela")
    )
)

pdf = df_consulta.toPandas()

mapa_meses = {
    1: 'Jan', 2: 'Fev', 3: 'Mar', 4: 'Abr', 5: 'Mai', 6: 'Jun',
    7: 'Jul', 8: 'Ago', 9: 'Set', 10: 'Out', 11: 'Nov', 12: 'Dez'
}
pdf['mes_nome'] = pdf['mes_num'].map(mapa_meses)

sns.set_theme(style="whitegrid")

g = sns.catplot(
    data=pdf,
    x="mes_nome",
    col="Estrela",
    kind="count",
    col_wrap=1,
    height=2.5, 
    aspect=3.0,
    hue="Estrela",
    palette="viridis",
    order=mapa_meses.values(),
    sharex=False
)

g.fig.suptitle("Sazonalidade: Quais meses têm menos observações?", y=1.02, fontsize=16)
g.set_axis_labels("Mês", "Total de Observações")

plt.show()

In [0]:
# Observadores mais assíduos

from pyspark.sql import functions as F
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

df_consulta = (
    spark.table("fato_medicao").alias("f")
    .join(spark.table("dim_observador").alias("o"), F.col("f.id_observador") == F.col("o.id"))
    .groupBy("o.sigla")
    .agg(F.count("f.id").alias("observacoes"))
    .orderBy(F.desc("observacoes"))
    .limit(10)
)

pdf_ranking = df_consulta.toPandas()

sns.set_theme(style="whitegrid")
plt.figure(figsize=(10, 8))

ax = sns.barplot(
    data=pdf_ranking,
    x="observacoes",
    y="sigla",         
    palette="viridis"
)

ax.bar_label(ax.containers[0], padding=3, fmt='%d')

plt.title("Observadores Mais Ativos", fontsize=16)
plt.xlabel("Quantidade de Medições")
plt.ylabel("Sigla do Observador")

sns.despine(left=True, bottom=True)

plt.show()

display(spark.table("dim_observador").count())